In [91]:
import pandas as pd
import numpy as np
import datetime
import warnings

warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline  

pd.set_option('display.max_columns', None)
from feature_engine.imputation import RandomSampleImputer
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import chi2_contingency


In [92]:
data = pd.read_excel("creditcardmarketing.xlsx")
data.head()

,Customer Number,Offer Accepted,Reward,Mailer Type,Income Level,# Bank Accounts Open,Overdraft Protection,Credit Rating,# Credit Cards Held,# Homes Owned,Household Size,Own Your Home,Average Balance,Q1 Balance,Q2 Balance,Q3 Balance,Q4 Balance
0,1,No,Air Miles,Letter,High,1,No,High,2,1,4,No,1160.75,1669.0,877.0,1095.0,1002.0
1,2,No,Air Miles,Letter,Medium,1,No,Medium,2,2,5,Yes,147.25,39.0,106.0,78.0,366.0
2,3,No,Air Miles,Postcard,High,2,No,Medium,2,1,2,Yes,276.50,367.0,352.0,145.0,242.0
3,4,No,Air Miles,Letter,Medium,2,No,High,1,1,4,No,1219.00,1578.0,1760.0,1119.0,419.0
4,5,No,Air Miles,Letter,Medium,1,No,Medium,2,1,6,Yes,1211.00,2140.0,1357.0,982.0,365.0


In [99]:
data.shape

(18000, 17)

In [93]:
data.columns = list(map(lambda x: x.lower().replace("# ","").replace(" ","_"), data.columns))
data.columns

Index(['customer_number', 'offer_accepted', 'reward', 'mailer_type',
       'income_level', 'bank_accounts_open', 'overdraft_protection',
       'credit_rating', 'credit_cards_held', 'homes_owned', 'household_size',
       'own_your_home', 'average_balance', 'q1_balance', 'q2_balance',
       'q3_balance', 'q4_balance'],
      dtype='object')

In [94]:
data.dtypes

customer_number           int64
offer_accepted           object
reward                   object
mailer_type              object
income_level             object
bank_accounts_open        int64
overdraft_protection     object
credit_rating            object
credit_cards_held         int64
homes_owned               int64
household_size            int64
own_your_home            object
average_balance         float64
q1_balance              float64
q2_balance              float64
q3_balance              float64
q4_balance              float64
dtype: object

In [95]:
data.isna().sum()

customer_number          0
offer_accepted           0
reward                   0
mailer_type              0
income_level             0
bank_accounts_open       0
overdraft_protection     0
credit_rating            0
credit_cards_held        0
homes_owned              0
household_size           0
own_your_home            0
average_balance         24
q1_balance              24
q2_balance              24
q3_balance              24
q4_balance              24
dtype: int64

In [71]:
# Based on the first observation
# 1- customer_number is not needed
data.drop(["customer_number"],axis=1, inplace=True)
# 2- offer_accepted is the target based on the business requirements
# 3- bank_accounts_open, credit_cards_held, homes_owned, household_size can be categorical

In [97]:
# Since the amount of nan's is 120 total, compared to the total records of 18,000 I will proceed
# with using RandomSampleImputer to replace nan values with random sample data from the dataframe
imputer = RandomSampleImputer(random_state=42)
test = imputer.fit_transform(data)
test.isna().sum()
#TODO JUST DID THE IMPUTER, CONTINUE
# 1- Check for outliers in the numericals
# 2- After cleaning do correlation matrix on the numericals
# 3- Then run the CHI2 on the numericals
# 4- Categoricals???

customer_number         0
offer_accepted          0
reward                  0
mailer_type             0
income_level            0
bank_accounts_open      0
overdraft_protection    0
credit_rating           0
credit_cards_held       0
homes_owned             0
household_size          0
own_your_home           0
average_balance         0
q1_balance              0
q2_balance              0
q3_balance              0
q4_balance              0
dtype: int64

In [ ]:
# For further processing, we 
numericals = data.select_dtypes(np.number)
categoricals = data.select_dtypes("object")

In [83]:
# for col in data.select_dtypes(np.number).columns:
#     sns.countplot(x = col, hue = 'offer_accepted', data=data)
#     plt.xlabel('Score Value')
#     plt.ylabel('Count')
#     plt.title(f'Distribution of Target by # {col}')
#     plt.show()

In [73]:
data.groupby('bank_accounts_open')['offer_accepted'].value_counts()



bank_accounts_open  offer_accepted
1                   No                12916
                    Yes                 776
2                   No                 3778
                    Yes                 234
3                   No                  283
                    Yes                  13
Name: offer_accepted, dtype: int64

In [78]:
# Running chi2 for every numerical column with the target
for col in data.select_dtypes(np.number).columns:
    # Running chi2
    crosstab_table = pd.crosstab(data[col], data['offer_accepted'])
    # Perform the test
    stat, p_value, dof, expected = chi2_contingency(crosstab_table)
    # Print results
    print(f"column: {col}, p-value: {p_value}")

column: bank_accounts_open, p-value: 0.5787148519062763
column: credit_cards_held, p-value: 0.7191405379088973
column: homes_owned, p-value: 0.8816359414526597
column: household_size, p-value: 0.0004550923324748873
column: average_balance, p-value: 0.7309110468764891
column: q1_balance, p-value: 0.9424607799831002
column: q2_balance, p-value: 0.8608690297459688
column: q3_balance, p-value: 0.15099200587134132
column: q4_balance, p-value: 0.020567701027427384


In [62]:
# Observations
# Clients with 3 bank accounts has a 4.39% chance to accept a credit card
# Clients with 2 bank accounts has a 5.83% chance to accept a credit card
# Clients with 1 bank accounts has a 5.67% chance to accept a credit card

In [49]:
data.bank_accounts_open.value_counts()

1    13692
2     4012
3      296
Name: bank_accounts_open, dtype: int64

In [50]:
data.credit_cards_held.value_counts()

2    7944
1    6154
3    3387
4     515
Name: credit_cards_held, dtype: int64

In [46]:
data.isna().sum()

customer_number          0
offer_accepted           0
reward                   0
mailer_type              0
income_level             0
bank_accounts_open       0
overdraft_protection     0
credit_rating            0
credit_cards_held        0
homes_owned              0
household_size           0
own_your_home            0
average_balance         24
q1_balance              24
q2_balance              24
q3_balance              24
q4_balance              24
dtype: int64

In [38]:
data.describe()

,bank_accounts_open,credit_cards_held,homes_owned,household_size,average_balance,q1_balance,q2_balance,q3_balance,q4_balance
count,18000.000000,18000.000000,18000.000000,18000.000000,17976.000000,17976.000000,17976.000000,17976.000000,17976.000000
mean,1.255778,1.903500,1.203444,3.499056,940.515562,910.450656,999.392190,1042.033600,810.185803
std,0.472501,0.797009,0.427341,1.114182,350.297837,620.077060,457.402268,553.452599,559.001365
min,1.000000,1.000000,1.000000,1.000000,48.250000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000,3.000000,787.500000,392.750000,663.000000,633.000000,363.000000
50%,1.000000,2.000000,1.000000,3.000000,1007.000000,772.000000,1032.000000,945.500000,703.000000
75%,1.000000,2.000000,1.000000,4.000000,1153.250000,1521.000000,1342.000000,1463.000000,1212.000000
max,3.000000,4.000000,3.000000,9.000000,3366.250000,3450.000000,3421.000000,3823.000000,4215.000000


customer_number          0
offer_accepted           0
reward                   0
mailer_type              0
income_level             0
bank_accounts_open       0
overdraft_protection     0
credit_rating            0
credit_cards_held        0
homes_owned              0
household_size           0
own_your_home            0
average_balance         24
q1_balance              24
q2_balance              24
q3_balance              24
q4_balance              24
dtype: int64

customer_number           int64
offer_accepted           object
reward                   object
mailer_type              object
income_level             object
bank_accounts_open        int64
overdraft_protection     object
credit_rating            object
credit_cards_held         int64
homes_owned               int64
household_size            int64
own_your_home            object
average_balance         float64
q1_balance              float64
q2_balance              float64
q3_balance              float64
q4_balance              float64
dtype: object

In [ ]:
for col in data.select_dtypes(np.number).columns:
    sns.boxplot(data.select_dtypes(np.number)[col])
    plt.show()